In [121]:
from transformers import GPT2LMHeadModel, GPT2Config
from gpt import GPTConfig, GPT
from torch.nn import functional as F

In [25]:
import torch
import tiktoken

In [122]:
pretrained = torch.load("model_19072.pt", map_location=torch.device('cpu'))

In [123]:
model = GPT(GPTConfig(vocab_size=50304))

In [124]:
checkpoint = torch.load("model_19072.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [125]:
enc = tiktoken.get_encoding("gpt2")

In [126]:
device = torch.device('cpu')
device_type = 'cpu'

In [127]:
model.eval()
num_return_sequences = 3
max_length = 50
tokens = enc.encode("Human: Can you tell me what are the best places to visit in USA?\n\nAssistant:")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
            logits, loss = model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)
    print('---------------------------------')

Human: Can you tell me what are the best places to visit in USA?

Assistant: I can visit you there.
Can you show me the Best Places to Visit in USA?
Assistant: Yes, I can show you the best
---------------------------------
Human: Can you tell me what are the best places to visit in USA?

Assistant: If a friend was to sit on my back, would you like a place for him? Would you like the place where he sat?
Student:
---------------------------------
Human: Can you tell me what are the best places to visit in USA?

Assistant: How many years from the time of your father’s death can you go to a place now without making a trip?
I: I always
---------------------------------


In [128]:
model.eval()
num_return_sequences = 3
max_length = 50
tokens = enc.encode("Human: Construct a list of tips to help reduce stress.\n\nAssistant:")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
            logits, loss = model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)
    print('---------------------------------')

Human: Construct a list of tips to help reduce stress.

Assistant: How to handle yourself.
Hate: Help others.
The following are ideas the teacher or a peer may use to help them identify their own emotional reactions to stress
---------------------------------
Human: Construct a list of tips to help reduce stress.

Assistant: “How much do you need to take into your house to be safe?” (I would suggest you start and keep track of the amount of chores)

---------------------------------
Human: Construct a list of tips to help reduce stress.

Assistant: Dr. Sankaran (firstname.lastname@example.org)
Ages: 16 - 25
Adolescence: 12 -19 years
Health
---------------------------------


In [38]:
GPTConfig()

GPTConfig(block_size=1024, vocab_size=50257, n_layer=12, n_head=12, n_embd=768)

In [47]:
hf_model = GPT2LMHeadModel(config=GPT2Config(vocab_size=50304))

In [ ]:
hf_model.load_state_dict(checkpoint['model'])

In [62]:

def transpose_weights(checkpoint):
    for key in checkpoint.keys():
        if "weight" in key and checkpoint[key].dim() == 2 :
            checkpoint[key] = checkpoint[key].t()
    return checkpoint

# def transpose_specific_weights(checkpoint):
#     for key in checkpoint.keys():
#         if key in ["transformer.wte.weight", "transformer.wpe.weight", "lm_head.weight"]:
#             checkpoint[key] = checkpoint[key].t()
#     return checkpoint

# Transpose the weights
# transposed_wghts = transpose_weights(transpose_weights(checkpoint['model']))
transposed_wghts = transpose_weights(checkpoint['model'])

In [63]:
hf_model.load_state_dict(transposed_wghts)

<All keys matched successfully>

In [64]:
hf_model.save_pretrained("./custom_gpt2")

In [65]:
xgen

tensor([[12982,    25,  1867,   284,   466,   287,  8031,    30,   198, 48902,
            25,  1867,   314,   892,   481,   307,   262,  1388,  3580,    13,
           314,   836,   470,  4601,   284,   307,   994,   287,  8031,    11,
           655,   994],
        [12982,    25,  1867,   284,   466,   287,  8031,    30,   198, 48902,
            25,   632,   447,   247,    82,   257,  4950,    11,  4950,  1295,
            11,   314,   447,   247,   303,  1239,  1107,  7891,  6609,   881,
           286,   262],
        [12982,    25,  1867,   284,   466,   287,  8031,    30,   198, 48902,
            25,  8200,  5537,  4139,    11,  8200,  5537,  4139,    11,  8200,
          5537,  4139,    11,  8200,  5537,  4139,   318,   257,  1049, 14971,
           290,   257],
        [12982,    25,  1867,   284,   466,   287,  8031,    30,   198, 48902,
            25,  1867,   466,   345,   466,   618,   345,  3067,    30,   198,
           464,  1708,   318,  5220,  1321,   618,  3067,  

In [110]:
tokens = enc.encode("Hello, I'm a scientist and work on")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
max_length = 50

In [111]:
while xgen.size(1) < max_length:
    logits = hf_model.forward(xgen)[0][:, -1, :] # (B, vocab_size)
    # get the probabilities
    probs = F.softmax(logits, dim=-1)
    # do top-k sampling of 50 (huggingface pipeline default)
    # topk_probs here becomes (5, 50), topk_indices is (5, 50)
    topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
    # select a token from the top-k probabilities
    # note: multinomial does not demand the input to sum to 1
    ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
    # gather the corresponding indices
    xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
    # append to the sequence
    xgen = torch.cat((xgen, xcol), dim=1)

In [112]:
for i in range(4):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)

Hello, I'm a scientist and work on the hand. For
’s at the level.
… go? And the same. in everything.
the world when the. I happen what.
I. for in
what.
Hello, I'm a scientist and work on for an ever so many who, it all those to be used to work, the same things on earth as it is to have that no is how is.
A scientist,
“ (1
Hello, I'm a scientist and work on a man with a lot more than is.
This won’t the sea to the name and that can win!
Follow it with, can or is only the follow through there�t an
Hello, I'm a scientist and work on the to and from . . It really it are and have your thing and , and then the one, and one by so. One - say that was a way here or on the world- that were


In [97]:
print(decoded)

Hello, I'm a language model, and the kids are all talking to each other on the topic you mentioned: it's just a language. We get the idea that our children should be the same as any other children and the same as any other


In [81]:
outputs = hf_model.generate(xgen, max_new_tokens=32)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
